<a href="https://colab.research.google.com/github/kaspergroenbek98/first-first-year-project/blob/master/chaosBook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/kaspergroenbek98/first-first-year-project.git

fatal: destination path 'first-first-year-project' already exists and is not an empty directory.


In [2]:
import numpy as np
import nltk
import csv
import nltk
import re

nltk.download('punkt')
nltk.download('stopwords')
stopwords = set(nltk.corpus.stopwords.words('english'))


# Read the data into a header and a data np.array - the array is then shuffled
with open('first-first-year-project/data.csv', encoding='latin1') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    data = np.array([np.array(line) for line in csv_reader])
header, data = data[0,:], data[1:,:]
print(header)
np.random.shuffle(data)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['other_topic' 'resolution_topics' 'gender' 'name' 'Resolution_Category'
 'retweet_count' 'text' 'tweet_coord' 'tweet_created' 'tweet_date'
 'tweet_id' 'tweet_location' 'tweet_state' 'user_timezone' 'tweet_region']


In [3]:
# Identify hashtags and insert them in the 15th column
print(data.shape)
col = np.zeros((data.shape[0],1), 'str')
data = np.append(data, col, axis=1)
print(data.shape) # Note that shape starts at 1 and ends at 16, so the interval is [0:15]

for i, text in enumerate(data[:,6]):
    results = re.findall(r"#\w+", text) # Finds matches and returns them as an iterable
    if results:
        data[i,15] = ' '.join(results)
    else:
        data[i,15] = ''
hashtags = set(hashtag for row in data[:,15] for hashtag in row.split())
print(hashtags)
print(len(hashtags)) # there are 1735 unique hashtags

(5011, 15)
(5011, 16)
{'#femdom', '#mickey', '#youneverknow', '#discover', '#DCIM', '#famouspeople', '#rachelclarkgiveaway', '#publishers', '#submissive', '#iaedchat', '#crossitoffmylist', '#AimHigh', '#nodebtdiet', '#joy', '#WeightLossJourney', '#NewYearsWish', '#NYE2015', '#Dance', '#ParksandRec', '#vegan', '#gaspump', '#getfit2015', '#nope', '#weightlossjourney', '#pets', '#dieting', '#bakewell', '#HR', '#NewYorkYankees', '#Xort', '#timeformyself', '#timetochange', '#TheOffice', '#thenewtriplethreat', '#winterweight', '#dreamofphilanthropy', '#ATL', '#London', '#sucker', '#I', '#weightlosstransformation', '#oops', '#TiredOfYourShit', '#pun', '#watch', '#MasBitch', '#feelsgood', '#sillybuttrue', '#Friends', '#2', '#Bucketlist', '#FeministN', '#loseweight', '#Bible', '#languagelearning', '#grateful', '#workouthard', '#math', '#hiphop', '#ResolutionBroken', '#follow', '#TalentPool', '#unfollow', '#insanitymax30', '#MagnumPI', '#IndieMusic', '#2015', '#GOBIG', '#ForSure', '#slurpslurp',

In [0]:
porter = nltk.PorterStemmer()

# Remove all stopwords, non-alphabet words (except spaces), and stem the words
for i, row in enumerate(data[:,6]):
    row = row.lower()
    row = ''.join(char for char in row if char.isalpha() or char == ' ')
    row = ' '.join(porter.stem(word) for word in row.split() if word not in stopwords)
    data[i,6] = row

# Create a long string of ALL words, then token them, and create a frequency distribution
actualWords = ' '.join(token for sentence in data[:,6] for token in sentence.split())
#tokens = nltk.word_tokenize(actualWords)
tokens = actualWords.split()
fd = nltk.FreqDist(tokens)


In [0]:
word_features = [word for (word, count) in fd.most_common(2000)]

# Compares the words of a specfic tweet with the top 2000 most-used words
def tweet_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in tweet_words)
    return features

""" 
Creates tuples with a vector containing boolean values depending on whether
or not the word is in the tweet - along with the label of the tweet.
"""
size = 1000
featuresets = [(tweet_features(d.split()), c) for (d,c) in zip(data[:size,6], data[:size,2])] # column 6 is text data, column 2 is gender data
train_set, test_set = featuresets[size//2:], featuresets[:size//2]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [6]:
print(nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(10)

0.55
Most Informative Features
        contains(weight) = True           female : male   =      4.1 : 1.0
             contains(ù) = True           female : male   =      4.0 : 1.0
          contains(come) = True             male : female =      3.9 : 1.0
          contains(much) = True           female : male   =      3.6 : 1.0
         contains(everi) = True           female : male   =      3.5 : 1.0
         contains(bitch) = True             male : female =      3.2 : 1.0
       contains(problem) = True             male : female =      3.2 : 1.0
         contains(watch) = True             male : female =      3.2 : 1.0
        contains(finish) = True             male : female =      3.2 : 1.0
        contains(resolv) = True             male : female =      3.2 : 1.0


In [0]:
bigrm = list(nltk.ngrams(tokens, 2))
fdbigrm = nltk.FreqDist(bigrm)


In [0]:
word_features = list(fdbigrm.keys())[:2000]

def tweet_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features[f'contains({word})'] = (word in tweet_words)
    return features
size = 1000
featuresets = [(tweet_features(nltk.ngrams(d.split(), 2)), c) for (d,c) in zip(data[:size,6], data[:size,2])] # column 6 is text data, column 2 is gender data
train_set, test_set = featuresets[size//2:], featuresets[:size//2]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [9]:
print(nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(10)

0.564
Most Informative Features
contains(('goal', 'newyearsresolut')) = True           female : male   =      2.8 : 1.0
contains(('newyearsresolut', 'stop')) = True           female : male   =      2.5 : 1.0
contains(('newyearsresolut', 'make')) = True             male : female =      2.5 : 1.0
contains(('year', 'new')) = True             male : female =      2.5 : 1.0
contains(('year', 'newyearsresolut')) = True             male : female =      2.3 : 1.0
contains(('resolut', 'stop')) = True           female : male   =      2.0 : 1.0
contains(('newyearsresolut', 'keep')) = True             male : female =      1.8 : 1.0
contains(('better', 'person')) = True             male : female =      1.8 : 1.0
  contains(('im', 'go')) = True             male : female =      1.8 : 1.0
contains(('resolv', 'eat')) = True             male : female =      1.8 : 1.0


In [10]:
'newyearsresolution' in stopwords

False